# Bike Rental Prediction

Bike-sharing systems provide people with a cheap option for commuting to work or taking a ride on a pleasant day. The popularity of bike riding has been increasing over the years as the problem of traffic jams is getting worse for other mediums of commute. It is an environment-friendly medium of commute. Besides, it has many health benefits. Nowadays, many businesses are providing bike-sharing services to customers. They need to predict the demand for bike rentals on a day. In this project, our objective is to predict bike rental count on a day based on various environmental and seasonal features. We have a dataset with data from the years 2011 and 2012. We want to build a regression model that will predict the demand for bike rentals on a given day.

**Problem type: regression**

### Team Members
- Manik Hossain

## Data Collection

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error

df = pd.read_csv('./dataset/day.csv')
df.head()

### Features Description
| Feature Name | Description | Type |
|--------------|-------------|------|
| instant      | Index of the record |
| dteday | Date |
| season | Season (1: spring, 2: summer, 3: fall, 4: winter) | Categorical |
| yr | Year (0 represents 2011, 1 represents 2012) | Categorical |
| mnth | Month (1 to 12) | Categorical |
| holiday | Whether the day is holiday or not | Categorical |
| weekday | Day of the week | Categorical |
| workingday | If the day is neither weekend nor holiday then it is 1, otherwise it is 0. | Categorical
| weathersit | Weather condition | Categorical |
| | 1. Clear, Few clouds, Partly cloudy, Partly cloudy. 
| | 2. Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist. 
| | 3. Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds. 
| | 4. Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog |
| temp | Normalized temperature in Celsius. | Numeric |
| atemp | Normalized feeling temperature in Celsius. | Numeric |
| hum | Normalized humidity. The values are divided by 100 (max) | Numeric |
| windspeed | Normalized wind speed. The values are divided to by (max) | Numeric |
| casual | count of casual users | Numeric |
| registered | count of registered users | Numeric |
| count | count of total rent bikes including both casual and registered | Numeric |

## Data Preparation

In [ ]:
df.drop('instant', axis=1, inplace=True)
df.rename(columns = {'cnt': 'count'}, inplace=True)
df.rename(columns = {'yr': 'year'}, inplace=True)
df.rename(columns = {'mnth': 'month'}, inplace=True)
df['year'] = pd.Categorical(df['year'])
df['month'] = pd.Categorical(df['month'].map({1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}))
df['holiday'] = pd.Categorical(df['holiday'])
df['weekday'] = pd.Categorical(df['weekday'].map({1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 0: 'Sunday'}))
df['workingday'] = pd.Categorical(df['workingday'])
df['weathersit'] = pd.Categorical(df['weathersit'].map({1: 'Clear', 2: 'Misty', 3: 'Light Snow/Rain', 4: 'Heavy Snow/Rain'}))
df['season'] = pd.Categorical(df['season'].map({1: 'Spring', 2: 'Summer', 3: 'Fall', 4: 'Winter'}))
df['dteday'] = pd.to_datetime(df['dteday'])
df.info()

In [ ]:
df.head()

## Exploratory Data Analysis

In [ ]:
print('Number of mismatch:', end=' ')
print(sum(df['casual'] + df['registered'] != df['count']))

There is no mismatch between total bike rental counts and the sum of different types of rental counts.

In [ ]:
sns.boxplot(y='count', data=df)
plt.ylabel('Bike rent count')
plt.show()

In [ ]:
df.describe()

There is no outlier for the `count` variable in the dataset.

In [ ]:
g = sns.relplot(
    data = df, x = 'dteday', y = 'count',
    hue='season', legend='brief', 
    alpha=1, height=8, aspect=1.5,
    palette='rocket'
)

plt.xticks(rotation=45)
plt.margins(0.02)
sns.move_legend(g, 'upper left', bbox_to_anchor=(0.1, 0.9))
plt.xlabel('Date')
plt.ylabel('Bike rent count')
plt.title('Bike rents over the years', pad=10)
plt.show()

In [ ]:
sns.barplot(
    data=df, x='season', y='count', 
    estimator=np.mean, palette='tab10',
    order=['Spring', 'Summer', 'Fall', 'Winter']
)
plt.ylabel('Bike rent count')
plt.xlabel("Season")
plt.title('Average bike rents in different seasons', pad=10)
plt.show()

In [ ]:
sns.barplot(
    data=df, x='weathersit', y='count', 
    estimator=np.mean, palette='tab10',
    order=['Clear', 'Misty', 'Light Snow/Rain']
)
plt.ylabel('Bike rent count')
plt.xlabel("Weather Condition")
plt.title('Average bike rents in different weather conditions', pad=10)
plt.show()

In [ ]:
sns.barplot(
    data=df, x='weekday', y='count', 
    estimator=np.mean, palette='tab10',
    order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)
plt.xticks(rotation=45)
plt.ylabel('Bike rent count')
plt.title('Average bike rents in different weekdays', pad=10)
plt.show()

In [ ]:
RdGn = sns.color_palette(["#FF0B04", "#8cdd49"])
sns.barplot(data=df, x='workingday', y='count', estimator=np.mean, palette=RdGn)
plt.ylabel('Bike rent count')
plt.xlabel("Is working day")
plt.xticks([0, 1], ['No', 'Yes'])
plt.title('Average bike rents in working days', pad=10)
plt.show()

In [ ]:
RdGn = sns.color_palette(["#FF0B04", "#8cdd49"])
sns.barplot(data=df, x='holiday', y='count', estimator=np.mean, palette=RdGn)
plt.ylabel('Bike rent count')
plt.xlabel("Is holiday")
plt.xticks([0, 1], ['No', 'Yes'])
plt.title('Average bike rents in holidays', pad=10)
plt.show()

In [ ]:
RdGn = sns.color_palette(["#FF0B04", "#8cdd49"])
sns.barplot(
    data=df, x='month', y='count', 
    order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
    estimator=np.mean, palette=RdGn
)
plt.ylabel('Bike rent count')
plt.xlabel("Month")
plt.title('Average bike rents in different months', pad=10)
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(df.corr(), square=True, annot=True, cbar=False, cmap='RdYlGn')
plt.title('Correlation Coefficients Matrix', pad=10)
plt.show()

We can drop either `temp` or `atemp` as these two variables are strongly correlated and they provide similar information. So keeping only one them is enough.

# Model Building

## Dataset Split

In [ ]:
data = df.drop(['dteday', 'temp', 'casual', 'registered'], axis=1)
data.info()

In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop('count', axis=1)
y = data['count']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.1,
    shuffle=False  # cannot shuffle timeseries data 
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train,
    test_size=0.11,
    shuffle=False
)

X_train.shape, X_val.shape, X_test.shape

## Preprocessing Pipeline

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = [cname for cname in X_train.columns if X_train[cname].dtype == 'category']
categorical_transformer = make_pipeline(OneHotEncoder(drop='first', handle_unknown='ignore'))

numeric_features = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]
numeric_transformer = make_pipeline(StandardScaler())

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_features),
        ('categorical', categorical_transformer, categorical_features)
    ])

## Linear Regression without reguralization (Baseline Model)

In [ ]:
baseline = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("model", LinearRegression())
    ])

In [ ]:
baseline.fit(X_train, y_train)
print(f"R-squared value on training data: {baseline.score(X_train, y_train):0.4f}")
print(f"R-squared value on validation data: {baseline.score(X_val, y_val):0.4f}")

The $r^2$ value on the training data is $0.8557$, which means $85.57\%$ of the variation in the dependent variable can be explained by the features. On the other hand, the $r^2$ value on the validation dataset is $0.1826$, which means $18.26\%$ of the variation in the dependent variable can be explained by the features. As we can see, our baseline model has a quite high variance.

In [ ]:
baseline_coef = baseline['model'].coef_
plt.plot(range(len(baseline_coef)), baseline_coef, 'rs')
plt.title('Coefficients in baseline model for different predictor variables')
plt.plot()

## ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNet

elastic_model = ElasticNet()

elastic_net = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("model", elastic_model)
    ])

In [ ]:
folds = 10
n = 20
l1_ratio = np.linspace(0.05, 1, n)
size = X_train.shape[0] // folds
mse_vals_elastic = np.zeros(n)

for i, l1 in enumerate(l1_ratio):
    mse = 0
    for k in range(5, folds):
        elastic_net["model"].l1_ratio = l1
        training_data = X_train[: k*size]
        training_label = y_train[: k*size]
        validation_data = X_train[k*size:(k+1)*size]
        validation_label = y_train[k*size:(k+1)*size]
        elastic_net.fit(training_data, training_label)
        validation_predict = elastic_net.predict(validation_data)
        mse += mean_absolute_error(validation_label, validation_predict)
    mse_vals_elastic[i] = mse / (folds - 5)

In [ ]:
elastic_net['model'].l1_ratio = l1_ratio[np.argmin(mse_vals_elastic)]
elastic_net.fit(X_train, y_train)
print(f"R-squared value on training data: {elastic_net.score(X_train, y_train):0.4f}")
print(f"R-squared value on validation data: {elastic_net.score(X_val, y_val):0.4f}")

In [ ]:
elastic_coef = elastic_net['model'].coef_
plt.plot(range(len(baseline_coef)), baseline_coef, 'rs', label="Baseline", alpha=0.8)
plt.plot(range(len(elastic_coef)), elastic_coef, 'bo', label="ElasticNet", alpha=0.8)
plt.title('Comparison of coefficients for different predictor variables')
plt.legend()
plt.plot()

As we can see from the plot, the values of coefficients haven't changed much after regularization.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_pred = baseline.predict(X_val)
print(f"MSE for baseline model: {mean_squared_error(y_val, y_pred)}")
print(f"MAE for baseline model: {mean_absolute_error(y_val, y_pred)}")

y_pred = elastic_net.predict(X_val)
print(f"MSE for elastic net model: {mean_squared_error(y_val, y_pred)}")
print(f"MAE for elastic net model: {mean_absolute_error(y_val, y_pred)}")

The error metrics have gotten worse for the regularized models. So, we should try non-linear models to improve the performance.

## Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor()
tree_regressor = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("model", tree)
    ])

folds = 10
n = 20
max_depths = np.arange(5, 15)
size = X_train.shape[0] // folds
mse_vals = np.zeros((len(max_depths)))

for i, depth in enumerate(max_depths):
    mse = 0
    for k in range(5, folds):
        tree_regressor["model"].max_depth = depth
        training_data = X_train[: k*size]
        training_label = y_train[: k*size]
        validation_data = X_train[k*size:(k+1)*size]
        validation_label = y_train[k*size:(k+1)*size]
        tree_regressor.fit(training_data, training_label)
        validation_predict = tree_regressor.predict(validation_data)
        mse += mean_squared_error(validation_label, validation_predict)
    mse_vals[i] = mse / (folds - 5)

In [ ]:
depth = max_depths[np.argmin(mse_vals)]
print(f"Best Hyperparameters: Max depth = {depth}")
tree_regressor['model'].max_depth = depth

In [ ]:
tree_regressor.fit(X_train, y_train)
print(tree_regressor.score(X_train, y_train))
print(tree_regressor.score(X_val, y_val))

Here, we have a negative $r^2$ value on the validation dataset. Hence, the decision tree model fits the data quite poorly.

## Next

We want to try out the following modeling techniques to improve our model performance:
- Ensemble Methods: AdaBoost 